In [23]:
import numpy as np
import pandas as pd

In [24]:
# Decorator to ensure double precision
def double_dec(func):
    def wrapper(*args):
        args = tuple(np.double(arg) for arg in args)

        result = func(*args)

        for element in result:
            if not isinstance(element, str):
                element = np.double(element)
        return result
    return wrapper

In [25]:
@double_dec
def simple_solver(a, b, c):

    delta = b**2 -4*a*c
    if delta < 0:
        return 'N/A', 'N/A'
    
    sq_delta = np.sqrt(delta)
    root1 = (-b + sq_delta) / (2 * a)
    root2 = (-b - sq_delta) / (2 * a)
    
    return root1, root2

In [26]:
coefficients = [
    (6, 5, -4),
    (6e154, 5e154, -4e154),
    (0, 1, 1),
    (1, -1e5, 1),
    (1, -4, 3.999999),
    (1e-155, -1e155, 1e155),
]

In [27]:
simple_solutions = [simple_solver(a, b, c) for a, b, c in coefficients]

sim_df = pd.DataFrame(coefficients, columns=["a", "b", "c"])
sim_df["Root 1"], sim_df["Root 2"] = zip(*simple_solutions)

print(sim_df)

               a              b              c       Root 1    Root 2
0   6.000000e+00   5.000000e+00  -4.000000e+00      0.50000 -1.333333
1  6.000000e+154  5.000000e+154 -4.000000e+154          inf      -inf
2   0.000000e+00   1.000000e+00   1.000000e+00          NaN      -inf
3   1.000000e+00  -1.000000e+05   1.000000e+00  99999.99999  0.000010
4   1.000000e+00  -4.000000e+00   3.999999e+00      2.00100  1.999000
5  1.000000e-155 -1.000000e+155  1.000000e+155          inf      -inf


/tmp/ipykernel_269/3453739440.py:4: RuntimeWarning: overflow encountered in scalar power
  delta = b**2 -4*a*c
/tmp/ipykernel_269/3453739440.py:4: RuntimeWarning: overflow encountered in scalar multiply
  delta = b**2 -4*a*c
/tmp/ipykernel_269/3453739440.py:9: RuntimeWarning: invalid value encountered in scalar divide
  root1 = (-b + sq_delta) / (2 * a)
/tmp/ipykernel_269/3453739440.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  root2 = (-b - sq_delta) / (2 * a)


In [32]:
@double_dec
def solver(a, b, c):
    # Normalize the coefficient
    nor_factor = max(abs(a), abs(b), abs(c))
    a, b, c = a/nor_factor, b/nor_factor, c/nor_factor

    # The min number of a double precision is about 1e-138. Abs smaller than this will underflow to zero
    min_range = 1e-308
    if abs(a) < min_range:
        if abs(b) < min_range:
            return 'No solution', 'No solution' if c != 0 else 'Any real number', 'Any real number' 
        else:
            return (-c / b), 'N/A'


    delta = b**2 -4*a*c
    if delta < 0:
        return 'N/A', 'N/A'
    
    sq_delta = np.sqrt(delta)
    
    # Square root of machine epslion is about 1e-8, so difference smaller than this should consider cancellation error
    tol = 2e-8
    if abs(sq_delta - b) < tol:
        root1 = (2*c) / (-b - sq_delta)
    else:
        root1 = (-b + sq_delta) / (2 * a)

    if abs(-b - sq_delta) < tol:
        root2 = (2*c) / (-b + sq_delta)
    else:
        root2 = (-b - sq_delta) / (2 * a)

    return root1, root2

In [35]:
solutions = [solver(a, b, c) for a, b, c in coefficients]

df = pd.DataFrame(coefficients, columns=["a", "b", "c"])
df["Root 1"], df["Root 2"] = zip(*solutions)

df.to_csv('approx solution.csv', index=False)
print(df)

               a              b              c       Root 1    Root 2
0   6.000000e+00   5.000000e+00  -4.000000e+00      0.50000 -1.333333
1  6.000000e+154  5.000000e+154 -4.000000e+154      0.50000 -1.333333
2   0.000000e+00   1.000000e+00   1.000000e+00     -1.00000       N/A
3   1.000000e+00  -1.000000e+05   1.000000e+00  99999.99999   0.00001
4   1.000000e+00  -4.000000e+00   3.999999e+00      2.00100     1.999
5  1.000000e-155 -1.000000e+155  1.000000e+155      1.00000       N/A
